In [ ]:
import os 

import pandas as pd
import numpy as np
import seaborn as sns

from utils import load_file
import matplotlib.pyplot as plt

In [ ]:
client_ids = ["0_0","0_1","0_2","0_3","0_4","0_5","1_0","1_1","1_2","1_3","1_4","1_5","2_0","2_1","2_2","2_3","2_4","2_5","3_0","3_1","3_2","3_3","3_4","3_5"]

In [ ]:
fed_learning_all_losses = pd.read_csv("losses/fed_learning_stats.csv")

In [ ]:
fig,axes = plt.subplots(nrows=4,ncols=6,figsize=(60, 30))
for client in client_ids:
    plot_id = client.split("_")
    fed_losses = fed_learning_all_losses.loc[fed_learning_all_losses['client_id'] == client].reset_index().drop(columns=['index','training_round','client_id','fed_train'])
    fed_losses = fed_losses.rename(columns={"validation_loss": "fed_val"})
    iso_losses = pd.read_csv("losses/"+"0_0"+".csv").drop(columns=["iso_train"])
    df = pd.concat([fed_losses, iso_losses], axis=1)
    df.plot(ax=axes[int(plot_id[0])][int(plot_id[1])],title=client)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
from torch.utils.data import Dataset, DataLoader

from models import ShallowNN

features = 197
batch_size = 64

In [113]:
from torch.autograd import grad
import time

from torch.autograd.functional import hessian

from utils import calculate_hessian

In [114]:
# Create an instance of the second model
model2 = ShallowNN(features)

# Load the pre-trained parameters (you should load your own trained model here)
model2.load_state_dict(torch.load('checkpt/_64_fedl_global.pth'))

validation_dataset = torch.load("testpt/0_0.pt")
validation_data_loader = DataLoader(validation_dataset, batch_size, shuffle = True)

loss_fn = torch.nn.L1Loss() 

In [116]:
hessian_mat, time = calculate_hessian(model2,loss_fn,validation_data_loader)

TypeError: The inputs given to hessian must be either a Tensor or a tuple of Tensors but the given inputs has type <class 'list'>.

In [ ]:
convert model.parameter to a tensor

In [81]:
hessian_mat.shape

torch.Size([14785, 14785])

In [83]:
2234/60

37.233333333333334

In [51]:
import torch
import time

# Create model
model = torch.nn.Sequential(torch.nn.Linear(1, 100), torch.nn.Tanh(), torch.nn.Linear(100, 1))

# Evaluate some loss on a random dataset
x = torch.rand((1000,1))
y = torch.rand((1000,1))
y_hat = model(x)
loss = ((y_hat - y)**2).mean()

print(loss)

''' Calculate Hessian '''
start = time.time()

# Allocate Hessian size
H = torch.zeros((num_param, num_param))

# Calculate Jacobian w.r.t. model parameters
J = torch.autograd.grad(loss, list(model.parameters()), create_graph=True)
J = torch.cat([e.flatten() for e in J]) # flatten

# Fill in Hessian
for i in range(num_param):
    result = torch.autograd.grad(J[i], list(model.parameters()), retain_graph=True)
    H[i] = torch.cat([r.flatten() for r in result]) # flatten

print(time.time() - start)

SyntaxError: invalid syntax (129499540.py, line 6)

In [24]:
H

tensor([[ 7.7808e-03, -9.2551e-05,  5.4254e-04,  ..., -3.1248e-03,
         -9.6760e-03,  1.4003e-02],
        [-9.2551e-05, -8.2589e-03, -4.0214e-04,  ...,  2.2377e-03,
          7.0062e-03, -1.0067e-02],
        [ 5.4254e-04, -4.0214e-04,  3.1513e-02,  ..., -1.2930e-02,
         -4.0726e-02,  5.8290e-02],
        ...,
        [-3.1248e-03,  2.2377e-03, -1.2930e-02,  ...,  1.0654e-01,
          2.9482e-01, -4.6080e-01],
        [-9.6760e-03,  7.0062e-03, -4.0726e-02,  ...,  2.9482e-01,
          8.4683e-01, -1.2899e+00],
        [ 1.4003e-02, -1.0067e-02,  5.8290e-02,  ..., -4.6080e-01,
         -1.2899e+00,  2.0000e+00]])

In [ ]:


# Get the shape of the model's parameters
model_parameters_shape = np.concatenate([p.data.view(-1).detach().cpu().numpy() for p in global_model.parameters()]).shape

# Sample data and labels (random data for illustration purposes)
inputs = torch.randn(100, input_dim)
labels = torch.randn(100, output_dim)

# Calculate the Hessian-vector product for both models
_, Hvp1 = fmin_l_bfgs_b(loss_and_hvp, x0=model1_parameters, args=(global_model, criterion, inputs, labels))
_, Hvp2 = fmin_l_bfgs_b(loss_and_hvp, x0=model2_parameters, args=(model2, criterion, inputs, labels))

# Compare the Hessian properties (e.g., eigenvalues, eigenvalue distribution, sensitivity analysis, etc.)
# You can use NumPy or other libraries to analyze the Hessian properties.